In [ ]:
from preprocess import *

mediaitems = extract_metadata("/Users/samyuktha/Downloads/Pixtale KL Photos")

In [ ]:


import json

with open("mediaitems.json", "r") as json_file:
    mediaitems = json.load(json_file)


mediaitems = generate_descriptions(dir_path, mediaitems)




In [ ]:
mediaitems.sort(key=lambda item: item["datetime"])

mediaitems

In [ ]:
import json
from geminiai import *

with open("mediaitems.json", "r") as json_file:
    mediaitems = json.load(json_file)


narration_script = get_script(mediaitems)

narration_script

In [ ]:
import shutil
from moviepy.editor import (
    VideoFileClip,
    ImageClip,
    concatenate_videoclips,
    TextClip,
    CompositeVideoClip,
    concatenate_audioclips,
    AudioFileClip,
)
from moviepy.video.fx.resize import resize
import os
from moviepy.video import fx as vfx
from moviepy.video.fx import speedx


clips = []

# Define a standard resolution
standard_resolution = (1080, 1920)  # Width x Height


os.makedirs("./audio")

audio_clips = []
# Process each scene
for i in range(len(narration_script["scenes"])):
    scene = narration_script["scenes"][i]

    text = scene["text"]

    filepath = os.path.join(dir_path, scene["media_source"]).lower()

    print(filepath)

    if os.path.exists(filepath):

        if not os.path.exists("./audio/" + str(i) + ".mp3"):

            synthesize_text(text, "./audio/" + str(i))

        audio_clip = AudioFileClip("./audio/" + str(i) + ".mp3")

        duration = audio_clip.duration

        if filepath.endswith((".mp4", ".mov")):
            clip = VideoFileClip(filepath)

            if duration < clip.duration:

                clip = clip.subclip(0, duration)
                
            clip=clip.resize(newsize=(standard_resolution[0], standard_resolution[1]))

        else:
            clip = ImageClip(filepath).set_duration(duration)

        # txt_clip = (
        #     TextClip(
        #         scene["narration"]["text"],
        #         color="white",
        #         font="Arial",
        #         fontsize=50,
        #         method="caption",
        #         stroke_width=2,
        #     )
        #     .set_position("bottom")
        #     .set_duration(float(duration))
        # )
        # clip = CompositeVideoClip([clip, txt_clip])
        clips.append(clip)

        audio_clips.append(audio_clip)


# shutil.rmtree("./audio")
# Concatenate all clips
final_clip = concatenate_videoclips(clips, method="compose")

final_audio_clip = concatenate_audioclips(audio_clips)

final_audio_clip.write_audiofile("final_trip_audio.mp3")

if final_clip.duration != final_audio_clip.duration:

    if final_clip.duration > final_audio_clip.duration:
        # Speed up the video clip to match the audio clip's duration
        speed_factor = final_clip.duration / final_audio_clip.duration
        final_clip = final_clip.fx(vfx.speedx, speed_factor)
    else:
        # Speed up the audio clip to match the video clip's duration

        speed_factor = final_audio_clip.duration / final_clip.duration
        print(speed_factor)
        cmd = (
            'ffmpeg -y -i final_trip_audio.mp3 -filter:a "atempo={}" output.mp3'.format(
                speed_factor
            )
        )

        os.system(cmd)


# Write the result to a file
final_clip.write_videofile("final_trip_video.mp4")

In [ ]:
cmd = "ffmpeg -y -i final_trip_video.mp4 -i output.mp3 -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac combined_video.mp4"
os.system(cmd)

In [ ]:
def list_voices():
    """Lists the available voices."""
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    # Performs the list voices request
    voices = client.list_voices()

    english_voices = []

    for voice in voices.voices:

        for language_code in voice.language_codes:
            if "en" in language_code:

                english_voices.append(
                    {
                        "language_codes": voice.language_codes[0],
                        "ssml_gender": texttospeech.SsmlVoiceGender(
                            voice.ssml_gender
                        ).name,
                        "name": voice.name,
                        "voice": "{}-{}".format(
                            voice.name,
                            texttospeech.SsmlVoiceGender(voice.ssml_gender).name,
                        ),
                    }
                )

    return english_voices

print(list_voices())

In [ ]:
from utils import *

public = "./static/public"
with open("data/narration_script.json", "r") as json_file:
    narration_script = json.load(json_file)

create_video(narration_script,public)

In [1]:
import json

from utils import *


with open("data/mediaitems.json", "r") as file:
    mediaitems = json.load(file)

create_video(mediaitems, "./static/public")

MoviePy - Writing audio in ./static/public/result/final_audio.mp3


MoviePy - Done.
Moviepy - Building video ./static/public/result/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./static/public/result/final_video.mp4



Moviepy - Done !
Moviepy - video ready ./static/public/result/final_video.mp4


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --e